## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys

# Import API Key
sys.path.insert(0, 'C:/Users/oscar/OneDrive/Data Analytics Bootcamp/Misc/')
from config import kgmaps as g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
imported_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df = imported_df.drop(['Unnamed: 0'], axis=1)
city_data_df.head(3)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hobart,AU,-42.8794,147.3294,49.86,83,40,11.50,scattered clouds
1,Qinzhou,CN,21.9500,108.6167,80.96,77,78,5.32,broken clouds
2,Ponta Do Sol,PT,32.6667,-17.1000,79.43,84,39,3.78,light rain


In [27]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Request the user to input a range of min/max temperatures he would desire for his vacation trip
min_temp = float(input("What is the minimum temperature (°F) you would like for your trip? "))
max_temp = float(input("What is the maximum temperature (°F) you would like for your trip? "))

What is the minimum temperature (°F) you would like for your trip? 75
What is the maximum temperature (°F) you would like for your trip? 90


In [28]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
filtered_cities_df.head(3)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,Qinzhou,CN,21.9500,108.6167,80.96,77,78,5.32,broken clouds
2,Ponta Do Sol,PT,32.6667,-17.1000,79.43,84,39,3.78,light rain
8,Kapaa,US,22.0752,-159.3190,77.32,85,75,8.05,broken clouds


In [29]:
# 4a. Determine if there are any empty rows.
filtered_cities_df.isnull().sum()

City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [30]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = filtered_cities_df.dropna()
preferred_cities_df.isnull().sum()

City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [31]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(3)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Qinzhou,CN,80.96,broken clouds,21.9500,108.6167,
2,Ponta Do Sol,PT,79.43,light rain,32.6667,-17.1000,
8,Kapaa,US,77.32,broken clouds,22.0752,-159.3190,


In [67]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
 
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Print current city
    city_name = row["City"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        print(f"Obtaining a hotel in {city_name}")
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print(f"No nearby hotel found for {city_name}")
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        

Obtaining a hotel in Qinzhou
Obtaining a hotel in Ponta Do Sol
Obtaining a hotel in Kapaa
Obtaining a hotel in Cabo San Lucas
Obtaining a hotel in Fort-Shevchenko
Obtaining a hotel in Bethel
Obtaining a hotel in Muros
Obtaining a hotel in Hithadhoo
Obtaining a hotel in Miyazu
Obtaining a hotel in Butaritari
Obtaining a hotel in Tarnos
Obtaining a hotel in Kattivakkam
Obtaining a hotel in Hilo
Obtaining a hotel in Georgetown
Obtaining a hotel in Mersing
Obtaining a hotel in Akyab
Obtaining a hotel in Faanui
Obtaining a hotel in Tevaitoa
Obtaining a hotel in Bengkulu
Obtaining a hotel in Buenavista
Obtaining a hotel in Sorong
Obtaining a hotel in Jand
Obtaining a hotel in Kavieng
Obtaining a hotel in Victoria
Obtaining a hotel in Salalah
Obtaining a hotel in Kudahuvadhoo
Obtaining a hotel in Saint George
Obtaining a hotel in Beloha
No nearby hotel found for Beloha
Obtaining a hotel in Atuona
Obtaining a hotel in Letlhakane
Obtaining a hotel in Banda Aceh
Obtaining a hotel in Vila Velha
O

In [18]:
### ADDITIONAL PIECE OF CODE
# This should be enabled for the very first run, to avoid requesting from google many times
# Save hotel DF in a CSV file
#hotel_df.to_csv('../Misc/Hotel_Database.csv')

# Additionally, backup the Hotel information
backup_df = hotel_df

In [10]:
# 7. Drop the rows where there is no Hotel Name.

# A: Print the number of rows and columns in the hotel DataFrame
shape_raw = hotel_df.shape

# B: Verify if any city missed an hotel 
missing_hotel_count = hotel_df['Hotel Name'].str.contains('NaN').sum()

# C: Get rid of rows where there is a missing hotel
hotel_clean_df = hotel_df[hotel_df["Hotel Name"].str.contains("NaN")==False]
shape_clean = hotel_clean_df.shape

print(f"From the {shape_raw[0]} cities in our vacation list, {missing_hotel_count} do not have hotel availability. Thus only {shape_clean[0]} will be considered for our vacations.")

From the 237 cities in our vacation list, 0 do not have hotel availability. Thus only 237 will be considered for our vacations.


In [2]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPY_vacation.csv'

# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

# For reference, this is a snippet of the WeatherPy Vacation map:
![WeatherPy_vacation_map](./WeatherPy_vacation_map.png)